# Maximizing Online Store Profits: A Data-driven A/B Testing Exploration

**Analyst : Ica Candra Rambadiana**<br>
*March 2023*<br>
<br>
**Reviewed by Warrie Warrie**<br>
*DSNai - Data Science Nigeria | Machine Learning Engineer | GIS Data Specialist*<br>

<div class="alert alert-block alert-success">
<b>OVERALL REVIEWER'S COMMENT V1</b> <a class="tocSkip"></a>

Congratulation Ica on completing your project!!👏 <br>
    
Your work has been brilliant and well detailed so far. You have shown great experience in analytics with efficient use of advanced pandas function. <br> I like the fact that you wer well specific and detailed with your summaries and conclusion<br> Keep it up 👏 <br>  For improvment, i have added comment in yellow that you can check out to make your future project better. <br>
    

Thank you and Good luck on your future projects 🥳

This project will conduct a list of hypotheses testing that may help **boost
revenue** in a big online store, together with the marketing
department.

### Objective
The analysis objective is:
- Assess and choose hypotheses that need to be prioritized
- Analyze and interpret the A/B testing results
- Decide whether we need to continue or stop the test

### STAGES
This analysis will be divided into three main sections.
 1. Get Started
 2. Priotizing Hypotheses
 3. A/B Testing Analysis<br>
    3.1. Data Preprocessing <br>
    3.2. EDA
            
            3.2.1. Cumulative Revenue
            3.2.2. Cumulative Average Order Size
            3.2.3. Conversion Rate
            3.2.4. Number of Orders per User
            3.2.5. Order Prices
            
    3.3. Hypothesis Testing<br>

## GET STARTED

In [1]:
#import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3, style='white')

import math as mth
import scipy.stats as st

import warnings
warnings.filterwarnings('ignore')

## PRIOTIZING HYPOTHESES

### Hypotheses

This section will use RICE and ICE, two commonly used frameworks for prioritizing hypotheses in the context of experimentation and A/B testing.

**Description**<br>
- `Hypotheses` — brief descriptions of the hypotheses
- `Reach` — user reach, on a scale of one to ten
- `Impact` — impact on users, on a scale of one to ten
- `Confidence` — confidence in the hypothesis, on a scale of one to ten
- `Effort` — the resources required to test a hypothesis on a scale of one to ten. The higher the Effort value, the more resource-intensive the test.

In [2]:
# load datasets
try:
    hypotheses = pd.read_csv('hypotheses_us.csv', sep=';')
except:
    hypotheses = pd.read_csv('/datasets/hypotheses_us.csv', sep=';')

In [3]:
# check datasets
hypotheses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Effort      9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes


In [4]:
# lowercase the columns name
pd.set_option('max_colwidth', 450)
hypotheses = hypotheses.rename(columns=str.lower)
hypotheses

,hypothesis,reach,impact,confidence,effort
0,Add two new channels for attracting traffic. This will bring 30% more users,3,10,8,6
1,Launch your own delivery service. This will shorten delivery time,2,5,4,10
2,Add product recommendation blocks to the store's site. This will increase conversion and average purchase size,8,3,7,3
3,Change the category structure. This will increase conversion since users will find the products they want more quickly,8,3,3,8
4,Change the background color on the main page. This will increase user engagement,3,1,1,1
5,Add a customer review page. This will increase the number of orders,3,2,2,3
6,Show banners with current offers and sales on the main page. This will boost conversion,5,3,8,3
7,Add a subscription form to all the main pages. This will help you compile a mailing list,10,7,8,5
8,Launch a promotion that gives users discounts on their birthdays,1,9,9,5


In [5]:
# evaluate the priority of hypotheses
hypotheses['ice'] = hypotheses[['impact', 'confidence']].prod(axis=1) / hypotheses['effort']
hypotheses['rice'] =hypotheses[['reach', 'impact', 'confidence']].prod(axis=1) / hypotheses['effort']

In [6]:
# top 5 ICE
hypotheses[['hypothesis', 'ice']].sort_values(by='ice', ascending=False).head()

,hypothesis,ice
8,Launch a promotion that gives users discounts on their birthdays,16.200000
0,Add two new channels for attracting traffic. This will bring 30% more users,13.333333
7,Add a subscription form to all the main pages. This will help you compile a mailing list,11.200000
6,Show banners with current offers and sales on the main page. This will boost conversion,8.000000
2,Add product recommendation blocks to the store's site. This will increase conversion and average purchase size,7.000000


In [7]:
# top 5 RICE
hypotheses[['hypothesis', 'rice']].sort_values(by='rice', ascending=False).head()


,hypothesis,rice
7,Add a subscription form to all the main pages. This will help you compile a mailing list,112.0
2,Add product recommendation blocks to the store's site. This will increase conversion and average purchase size,56.0
0,Add two new channels for attracting traffic. This will bring 30% more users,40.0
6,Show banners with current offers and sales on the main page. This will boost conversion,40.0
8,Launch a promotion that gives users discounts on their birthdays,16.2


### Insight

- ICE or RICE generated a different result of the hypothesis to be prioritized
- The highest score hypothesis based on ICE is `Launch a promotion that gives users discounts on their birthdays`
- The highest score hypothesis based on RICE is `Add a subscription form to all the main pages.`
- Ultimately, the decision to use RICE or ICE depends on the specific project and the goals we want to achieve.
- If we have limited resources and want to focus on relatively easy ideas to implement or based purely on their potential impact, then ICE might be a better fit.
- Meanwhile, If we are working on a project with a large user base and want to prioritize ideas that will impact a significant number of users, then RICE might be the better option.

## A/B TESTING ANALYSIS

### Data Preprocessing

After conducting the A/B testing, we got two primary datasets (i.e., `orders` and `visitors`). This section will ensure the datasets are free from missing values and duplicates, contain proper datatype, then transform the dataset for further analysis.

#### Orders

**Description:**
- `transactionId` — order identifier
- `visitorId` — identifier of the user who placed the order
- `date` — of the order
- `revenue` — from the order
- `group` — the A/B test group that the user belongs to

In [8]:
# load dataset
try:
    orders = pd.read_csv('orders_us.csv', parse_dates=['date'])
except:
    orders = pd.read_csv('/datasets/orders_us.csv', parse_dates=['date'])

In [9]:
# check `orders` info
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1197 non-null   int64         
 1   visitorId      1197 non-null   int64         
 2   date           1197 non-null   datetime64[ns]
 3   revenue        1197 non-null   float64       
 4   group          1197 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 46.9+ KB


In [10]:
# check duplicates
orders.duplicated().sum()

0

In [11]:
# ensure one `visitorId` belongs to one `group`
num_visitor_group = orders[['visitorId','group']].groupby('visitorId')['group'].nunique()
num_visitor_group.sort_values(ascending=False)

visitorId
2654030115    2
3891541246    2
3941795274    2
1404934699    2
351125977     2
             ..
1476256851    1
1476741193    1
1484903144    1
1485964287    1
4283872382    1
Name: group, Length: 1031, dtype: int64

In [12]:
# list `visitorId` that belongs to one group
one_group = num_visitor_group[num_visitor_group == 1].index

# separate `visitorId` that belongs to one group and two groups
orders_clean = orders.loc[orders['visitorId'].isin(one_group)]
visitor_two_groups = orders.loc[~orders['visitorId'].isin(one_group)]

In [13]:
print('raw orders rows:', len(orders))
print('Non-duplicated rows:', len(orders_clean))
print('Duplicated rows:', len(visitor_two_groups))
print('Number of unique visitorIds were in two groups:', visitor_two_groups.visitorId.nunique())

raw orders rows: 1197
Non-duplicated rows: 1016
Duplicated rows: 181
Number of unique visitorIds were in two groups: 58


In [14]:
# check the result
print('Total rows:', len(visitor_two_groups))
visitor_two_groups.sort_values(by=['visitorId', 'date']).head()

Total rows: 181


,transactionId,visitorId,date,revenue,group
71,3679129301,8300375,2019-08-01,165.7,B
703,4293855558,8300375,2019-08-07,30.5,A
246,437656952,199603092,2019-08-02,55.7,B
1102,1941707234,199603092,2019-08-12,100.0,A
26,2223239646,199603092,2019-08-15,55.7,A


**FINDING:**<br>
- The dataset has no missing values and duplicates
- There are `visitorId`s who are categorized into both groups (A and B)
- In A/B testing, it's important to ensure that each user is only assigned to one group. If a user is in both groups (A and B), then the results of the test will be skewed and invalid, as the user's behavior will be counted twice.

**ACTION:**<br>
- Here are the steps to overcome double group in `visitorId`
    1. Separate `visitorId` who is in the double group (✅done) 
    2. Choose the group of `visitorId` based on first order date 
    3. Remove `orders` records from another group
    4. Preserve `orders` records from the same group

In [15]:
#  Choose the group of `visitorId` based on first order date
visitor_two_groups_clean = (visitor_two_groups
                           .sort_values('date')
                           .groupby('visitorId')
                           .first()
                           .reset_index()
                           )

print('Total rows:', len(visitor_two_groups_clean))
visitor_two_groups_clean.head()

Total rows: 58


,visitorId,transactionId,date,revenue,group
0,8300375,3679129301,2019-08-01,165.7,B
1,199603092,437656952,2019-08-02,55.7,B
2,232979603,2670069237,2019-08-31,45.6,B
3,237748145,3357467820,2019-08-12,10.2,A
4,276558944,3894437543,2019-08-12,80.7,A


In [16]:
# Remove `orders` records from another group
# Preserve `orders` records from the same group
visitorId_clean = (visitor_two_groups
                   .merge(visitor_two_groups_clean,
                          how='inner',
                          on=['visitorId', 'group'],
                          suffixes=['_','_ref'])
                   .sort_values(by=['visitorId','date_'])
                   .drop(columns=['transactionId_ref', 'date_ref', 'revenue_ref'])
                  )

visitorId_clean.columns = ['transactionId', 'visitorId', 'date', 'revenue', 'group']

# check the result
print('Total rows:', len(visitorId_clean))
visitorId_clean.head(8)

Total rows: 100


,transactionId,visitorId,date,revenue,group
5,3679129301,8300375,2019-08-01,165.7,B
34,437656952,199603092,2019-08-02,55.7,B
89,2670069237,232979603,2019-08-31,45.6,B
4,3357467820,237748145,2019-08-12,10.2,A
1,2594268419,237748145,2019-08-15,20.8,A
2,3702128013,237748145,2019-08-20,20.8,A
3,972407388,237748145,2019-08-27,10.2,A
92,3894437543,276558944,2019-08-12,80.7,A


In [17]:
# merge 'visitorId_clean` and `orders_clean`
orders_clean = pd.concat([orders_clean, visitorId_clean])

# retrieve the result
orders_clean

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B
5,182168103,935554773,2019-08-15,35.0,B
...,...,...,...,...,...
16,3700356838,4256040402,2019-08-20,275.5,A
15,1610091760,4256040402,2019-08-23,265.8,A
17,3402467119,4256040402,2019-08-27,30.1,A
18,1415970216,4256040402,2019-08-31,80.5,A


In [18]:
# validate the results
# ensure one `visitorId` only belongs to one `group`
check = orders_clean.groupby('visitorId')['group'].nunique().reset_index()
check['group'].nunique()

1

#### Visitors

**Descriptions:**<br>
- `date` — date
- `group` — A/B test group
- `visits` — the number of visits on the date specified in the A/B test group specified


In [19]:
# load dataset
try:
    visitors = pd.read_csv('visitors_us.csv', parse_dates=['date'])
except:
    visitors = pd.read_csv('/datasets/visits_us.csv', parse_dates=['date'])

In [20]:
# check info
visitors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    62 non-null     datetime64[ns]
 1   group   62 non-null     object        
 2   visits  62 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.6+ KB


In [21]:
# retrieve datasets
visitors.head()

,date,group,visits
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


In [22]:
# check duplicates
visitors.duplicated(['date', 'group']).sum()

0

#### Cumulative Data

This section will aggregate and transform all the datasets to contain cumulative values for total transactions, buyers, visitors, and revenue.

In [23]:
# initialize data frame
## aggregate data based on `date` and `group`
dates_groups = orders_clean[['date', 'group']].drop_duplicates()

# aggregate and count parameters in daily orders from both groups
cum_orders = (dates_groups
             .apply(lambda df: orders_clean
                               .loc[np.logical_and(orders_clean['date'] <= df['date'],
                                                   orders_clean['group'] == df['group'])]
                               .agg({'date' : 'max',
                                     'group' : 'max',
                                     'transactionId' : 'nunique',
                                     'visitorId' : 'nunique',
                                     'revenue' : 'sum'})
                                ,axis=1
                   )
             .sort_values(by=['date','group'])
            )

                                                          
cum_orders

,date,group,transactionId,visitorId,revenue
55,2019-08-01,A,24,20,2356.8
66,2019-08-01,B,21,20,1620.0
175,2019-08-02,A,43,37,3825.1
173,2019-08-02,B,45,43,4243.8
291,2019-08-03,A,67,61,5640.3
...,...,...,...,...,...
533,2019-08-29,B,559,519,80971.1
757,2019-08-30,A,506,463,58057.7
690,2019-08-30,B,582,542,84328.6
958,2019-08-31,A,516,471,58997.3


In [25]:
# aggregate and count cumulative visitors by date and group
cum_visitors = (dates_groups
               .apply(lambda df: visitors
                                 .loc[np.logical_and(visitors['date']<=df['date'],
                                                     visitors['group']==df['group'])]
                                 .agg({'date':'max',
                                       'group':'max',
                                       'visitors':'sum'})
                      ,axis=1
                     )
               .sort_values(by=['date','group'])
              )

cum_visitors

SpecificationError: Column(s) ['visitors'] do not exist

In [ ]:
# merge `cum_orders` and `cum_vistors`
cum_data = cum_orders.merge(cum_visitors, on=['date', 'group'])
cum_data.columns = ['date', 'group', 'orders' ,'buyers' , 'revenue', 'visitors']

cum_data

In [ ]:
# check date range
cum_data['date'].describe()

**Note:** The A/B test was held for 31 days, during August 2019

In [ ]:
# check sample size
print('Sample size for group A:',
      cum_data[cum_data['group']=='A']['visitors'].sum())
print('Sample size for group B:',
      cum_data[cum_data['group']=='B']['visitors'].sum())

**Note:** The A/B test was held for 31 days during August 2019

#### Cumulative Data for Group A

In [ ]:
# separate `cum_data` group A
cum_data_a = cum_data.query("group == 'A'")

print('Total rows:', len(cum_data_a))
cum_data_a.head()

#### Cumulative Data for Group B

In [ ]:
# separate `cum_data` group A
cum_data_b = cum_data.query("group == 'B'")

print('Total rows:', len(cum_data_b))
cum_data_b.head()

### EDA

This section will compare several metrics from the control group (A) and treatment group (B), which are associated with revenue.

#### Cumulative Revenue

Comparison of cumulative revenue by the group during August 2019.

In [ ]:
# plot cumulative revenue by group
plt.figure(figsize=(15,5))
plt.title('Cumulative Revenue by Group')
sns.lineplot(data=cum_data,
             x='date',
             y='revenue',
             hue='group'
            );

In [ ]:
# generate a function to graph the cumulative difference between both groups

def cumulative_difference (control, treatment, variable):
    
    # merge variable column of group_a and group_b
    mergeData = control[['date', variable]].merge(treatment[['date', variable]],
                                                   on='date',
                                                   how='left')
    mergeData.columns = ['date', 'A', 'B']
    
    # calculate the relative difference
    mergeData['rel_diff'] = mergeData['B']/mergeData['A']-1

    # plot cumulative revenue by group
    plt.figure(figsize=(15,5))
    plt.title(f'Relative difference {variable} by group')
    sns.lineplot(data=mergeData,
                 x='date',
                 y='rel_diff',
                 color='orange'
                )
    plt.axhline(y=0, color='r', linestyle='--')
    plt.show()

In [ ]:
# get the cumulative difference of `revenue`
cumulative_difference (cum_data_a, cum_data_b, 'revenue')

**FINDING:**<br>
- The cumulative revenue of the Blue group (A, control) is consistently lower than the Orange group (B, treatment)
- There's a surge in Group B toward the end of the test, followed by a stabilized trend until the test ended
- The B group consistently demonstrates better revenue than the A group, and the relative difference tends to grow over time.

**INSIGHT:**<br>
- The B group generated better revenue than the A group.
- If there's enough data to reach the significance level, we may as well stop the test

**RECOMMENDATION:**<br>
- It's highly advisable to check the anomaly causing the surge in Group B since It may bring bias to the result

#### Cumulative Average Order Size
- Graph of cumulative average order size by group
- Graph of the relative difference in cumulative average order size for group B compared with group

In [ ]:
# calculate `order_size`
## cum_data
cum_data['order_size'] = cum_data['revenue'] / cum_data['orders']

cum_data[['date', 'group', 'revenue', 'orders', 'order_size']].head(3)

In [ ]:
# plot cumulative average order size by group
plt.figure(figsize=(15,5))
plt.title('Cumulative Average Order Size by Group')
sns.lineplot(data=cum_data,
             x='date',
             y='order_size',
             hue='group'
            );

In [ ]:
# calculate `order_size`
## cum_data a
cum_data_a['order_size'] = cum_data_a['revenue'] / cum_data_a['orders']

## cum_data b
cum_data_b['order_size'] = cum_data_b['revenue'] / cum_data_b['orders']

In [ ]:
# generate the cumulative difference of `order_size`
cumulative_difference (cum_data_a, cum_data_b, 'order_size')

**FINDING:**<br>
- The cumulative `order_size` of group A and B fluctuate until the mids month, then group B is consistently higher than group A
- There's a surge in group B toward the end of the test, followed by a downtrend until the trial ended
- Meanwhile, group A shows a slight uptrend toward the end of the test
- Most of  the time, the B group demonstrates better average order size than the A group
- Group B ever had a lower average order size than group A in the mids month

**INSIGHT:**<br>
- The B group generated a better average order size than the A group
- However, there's a slightly changing trend toward the end of the test (i.e., the opposite tendencies of both groups). It may indicate stabilization or position-shifting possibilities
- If there's enough data to reach the significance level, we may as well stop the test

**RECOMMENDATION:**<br>
- It's advisable to check the anomaly causing the surge in Group B and keep monitoring whether there are shifting changes from both groups

#### Conversion Rate

- Calculating each group's conversion rate (ratio of orders to the number of visits for each day)
- Plotting the daily conversion rates of the two groups and describe the difference

In [ ]:
# calculate conversion rate
cum_data['conversion_rate'] = cum_data['orders'] / cum_data['visitors']

cum_data[['date', 'group', 'orders', 'visitors', 'conversion_rate']].head(3)

In [ ]:
# plot cumulative average order size by group
plt.figure(figsize=(15,5))
plt.title('Cumulative Conversion Rate by Group')
sns.lineplot(data=cum_data,
             x='date',
             y='conversion_rate',
             hue='group'
            )
plt.show()

In [ ]:
# calculate conversion rate
## cum_data_a
cum_data_a['conversion_rate'] = cum_data_a['orders']/cum_data_a['visitors']

## cum_data_b
cum_data_b['conversion_rate'] = cum_data_b['orders']/cum_data_b['visitors']

In [ ]:
# plot the relative difference
cumulative_difference(cum_data_a, cum_data_b, 'conversion_rate')

**FINDING:**<br>
- The cumulative conversion rate of groups A and B fluctuated in the former period, then group B gained a higher conversion rate than group A consistently
- Both groups show slight downtrends
- The B group has demonstrated a better conversion rate than the A group consistently since the early test

**INSIGHT:**<br>
- The B group generated a better conversion rate than the A group
- If there's enough data to reach the significance level, we may as well stop the test

#### Number of Orders per User

- Plotting a scatter chart of the number of orders per user
- Calculating the 95th and 99th percentiles for the number of orders per user
- Defining the point at which a data point becomes an anomaly.

In [ ]:
# get the number of orders each user
## count orders from each unique `visitorId`

n_order_per_user = (orders_clean
                    .groupby('visitorId', as_index=False)
                    .agg({'transactionId':'nunique','group':'max'})
                    .rename(columns={'transactionId':'n_orders'})
                   )

n_order_per_user.tail()

In [ ]:
# check number of orders in general
n_order_per_user['n_orders'].describe()

In [ ]:
# check percentiles to detect outliers
n_order_per_user['n_orders'].quantile([0.95, 0.99])

In [ ]:
# plot distribution number of order per user
plt.figure(figsize=(4,3))
sns.catplot(data=n_order_per_user, y='n_orders', x='group', hue='group', palette=['orange','#9fc5e8'])

plt.axhline(y=n_order_per_user['n_orders'].quantile(0.95), color='g', linestyle='--')
plt.axhline(y=n_order_per_user['n_orders'].quantile(0.99), color='r', linestyle='--')

plt.show()

**INSIGHT:**<br>
- 95% of the customers (`visitorsId` who make order) create no more than two orders (<1.5 orders)
- The majority of customers only make one order
- Both groups contain outliers. These outliers can mask the different between two groups

**ACTION:**<br>
- We will filter `visitorId` who has an abnormal number of orders (>1.5 orders, based on quantile 0.95)
- The A/B testing analysis will use raw data vs filtered data and examine whether the outliers influence the result

In [ ]:
# filter outliers
usersManyOrders = (n_order_per_user
                   .loc[(n_order_per_user['n_orders'] > n_order_per_user['n_orders'].quantile(0.95))]
                   .reset_index(drop=True)
                  )

usersManyOrders.tail()

In [ ]:
usersManyOrders['n_orders'].describe()

#### Order Prices

- Plotting a scatter chart of order prices. 
- Calculating the 95th and 99th percentiles of order prices. 
- Defining the point at which a data point becomes an anomaly.

In [ ]:
# get the overview of `revenue` in general
orders_clean['revenue'].describe()

In [ ]:
# check the distribution
plt.figure(figsize=(8,3))
plt.title('Revenue Distribution')
sns.histplot(orders_clean[orders_clean['group']=='A']['revenue'], kde=True, label='A')
sns.histplot(orders_clean[orders_clean['group']=='B']['revenue'], kde=True, label='B', color='orange')

plt.axis([0,800, 0,200])
plt.legend()
plt.show()

In [ ]:
# check percentiles to detect outliers
orders_clean['revenue'].quantile([0.95, 0.99])

In [ ]:
# plot distribution number of order per user
plt.figure(figsize=(4,3))
sns.catplot(data=orders_clean, y='revenue', x='group', hue='group', palette=['orange','#9fc5e8'])

plt.axhline(y=orders_clean['revenue'].quantile(0.95), color='g', linestyle='--')
plt.axhline(y=orders_clean['revenue'].quantile(0.99), color='r', linestyle='--')

plt.show()

**FINDING:**<br>
- Both revenues from groups A and B seem to have a positively skewed distribution
- 95% of revenues were below 428.3 USD
- Both groups A and B contains outliers
- However, the super-extreme outlier was detected in group B

**INSIGHT:**<br>
- The surge in cumulative revenue and order size from group B is most likely due to the presence of super-extreme revenue
- It can make group B seems to be performing significantly better than group A when it may not be the case

**ACTION:**<br>
- We will filter revenue from expensive orders (> 428.3 USD, based on quantile 0.95)
- The A/B testing analysis will use raw data vs filtered data and examine whether the outliers influence the result

In [ ]:
# filter outliers
usersExpensiveOrders = (orders_clean
                        .loc[(orders_clean['revenue'] > orders_clean['revenue'].quantile(0.95))]
                        .reset_index(drop=True)
                       )

usersExpensiveOrders.tail()

In [ ]:
usersExpensiveOrders['revenue'].describe()

### Hypotheses Testing

This section will test the statistical significance of the difference in conversion rate and average order size between both groups. Before starting the hypotheses test, we will create datasets excluding anomalous users.

#### Preparation

##### Sample Preparation

We will take the following steps to transform the variable before conducting the analysis. <br>
1. Separates the interest column to be analysed into two variables based on its' groups
2. Concatenate the successes and non-successes values (trials-successes) into pd.Series. Use 0 as the non-success values
3. Use the pd.Series result as input in statistical analysis

#####  Anomalous Users

We will list the abnormal visitorId to be excluded in the subsequent analysis. 

In [ ]:
# retrieve abnormal n_orders
usersManyOrders.tail(2)

In [ ]:
# retrieve abnormal revenue
usersExpensiveOrders.tail(2)

In [ ]:
# merge abnormal `visitorId`
abnormalUsers = pd.concat([usersManyOrders['visitorId'], usersExpensiveOrders['visitorId']]).drop_duplicates()

# check the result
abnormalUsers.info()

In [ ]:
# exclude abnormal visitors from `orders_clean`
orders_no_outliers = (orders_clean
                      .loc[np.logical_not(orders_clean['visitorId'].isin(abnormalUsers))]
                     )

print('Total rows orders_clean:', len(orders_clean))
print('Total rows orders_no_outliers:', len(orders_no_outliers))
print('Removed rows:', len(orders_clean)-len(orders_no_outliers))

orders_no_outliers.head()

##### Function: Statistical Test

The statistical analysis will be carried out in two stages.<br>
1. Normality test<br>
We will use a two-proportion Z-test for boolean data and a Shapiro-Wilk test for comparing two means for continuous variable
2. Distribution test <br>
To know whether the A/B testing result is significantly different, we will use a t-test if the distribution of both samples is normal/bell-shaped or use the Mann-Whitney U test if the distributions are not normal.

The Test will be applied to test the conversion rate and average order size. 

In [ ]:
# normality test
## two-proportion z-test : test the equality of proportions

def proportion_test (control, treatment, alpha):
    
    successes = [control.sum(), treatment.sum()]
    trials = [len(control), len(treatment)]

    # success proportion Group A and Group B
    pA = successes[0]/trials[0]
    pB = successes[1]/trials[1]

    # success proportion in the combined groups
    p_combined = (successes[0]+successes[1]) / (trials[0]+trials[1])

    # difference between two groups proportion
    difference = pA - pB

    # calculate how far from 0 our statistic turned out to be
    ## standard deviations of the standard normal distribution
    z_value = difference/mth.sqrt(p_combined*(1 - p_combined) * (1/trials[0] + 1/trials[1]))

    # setting up the standard normal distribution (mean 0, standard deviation 1)
    distr = st.norm(0,1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-value: ', p_value)

    if (p_value < alpha):
        print("Rejecting the null hypothesis: there is a significant difference between the proportions")
    else:
        print("Failed to reject the null hypothesis: there is no reason to consider the proportions different")

In [ ]:
# normality test
## shapiro-wilk test: whether a continuous variable follows a normal distribution

def shapiro_wilk_test (control, treatment, alpha):
    
    results_control = st.shapiro(control)
    pvalue_control = results_control[1]

    results_treatment = st.shapiro(treatment)
    pvalue_treatment = results_treatment[1]
    #-------------------------------------------------------------------------------------
    print('p-value control: ', pvalue_control)

    if (pvalue_control < alpha):
        print("Control : Reject H0, the distribution is not normal")
    else:
        print("Control : Accept H0, the distribution seems to be normal") 
    #-------------------------------------------------------------------------------------------------
    print('p-value treatment: ', pvalue_treatment)

    if (pvalue_treatment < alpha):
        print("Treatment : Reject H0, the distribution is not normal")
    else:
        print("Treatment : Accept H0, the distribution seems to be normal") 

In [ ]:
# A/B test
## different test for two independet samples
## normal distribution

def mannwhitneyu_test (control, treatment, alpha):    

    mannwhitneyu = st.mannwhitneyu(control, treatment)[1]

    if mannwhitneyu < alpha :
        print('p-value:', mannwhitneyu)
        print('Reject H0: The distribution of the two groups is different significantly')
        print(f'relative difference: {treatment.mean()/control.mean()-1:.3f}')
    else:
        print('p-value:', mannwhitneyu)
        print('Accept H0: The distributions of the two groups are equal')
        print(f'relative difference: {treatment.mean()/control.mean()-1:.3f}')

In [ ]:
# A/B test
## different test for two independet samples
## non-normal distribution

def t_test_ind(control, treatment, alpha):
       
    t_stat, p_val = st.ttest_ind(control, treatment)
    
    if p_val < alpha:
        print('p-value:', p_val)
        print('Reject H0: The means of the two groups is different significantly')
    else:
        print('p-value:', p_val)
        print('Accept H0: The means of the two groups is equal')

#### Conversion Rate

1. Comparing the conversion rate between the groups using the raw data. 
2. Comparing the conversion rate between the groups using the filtered data

##### Raw Data

In [ ]:
# Seperate `orders` from group A
ordersByUsersA = (orders_clean[orders_clean['group']=='A']
                  .groupby('visitorId', as_index=False)
                  .agg({'transactionId' : pd.Series.nunique})
                 )

ordersByUsersA.columns = ['visitorId', 'orders']

#----------------------------------------------------------------------------------
# Seperate `orders` from group B
ordersByUsersB = (orders_clean[orders_clean['group']=='B']
                  .groupby('visitorId', as_index=False)
                  .agg({'transactionId' : pd.Series.nunique})
                 )

ordersByUsersB.columns = ['visitorId', 'orders']

In [ ]:
# create sample for group A and group B
## Group A
sampleA = (pd.concat([ordersByUsersA['orders'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='A']['visitors'].sum()
                                                -len(ordersByUsersA['orders'])
                                               ),
                                name='orders'
                               )
                     ], axis=0
                    )
          )

## Group B
sampleB = (pd.concat([ordersByUsersB['orders'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='B']['visitors'].sum()
                                                -len(ordersByUsersB['orders'])
                                               ),
                                name='orders'
                               )
                     ], axis=0)
          )

In [ ]:
# apply the two-proportion Z-test
proportion_test(sampleA, sampleB, 0.05)

**Note:** We will use the Mann-Whitney U test since the distribution of samples are not normal

In [ ]:
# apply mannwhitneyu test
mannwhitneyu_test(sampleA, sampleB, 0.05)

##### Filtered Data

In [ ]:
# Seperate `orders_no_outliers` from group A
ordersFilteredA = (orders_no_outliers[orders_no_outliers['group']=='A']
                  .groupby('visitorId', as_index=False)
                  .agg({'transactionId' : pd.Series.nunique})
                 )

ordersFilteredA.columns = ['visitorId', 'orders']

#----------------------------------------------------------------------------------
# Seperate `orders_no_outliers` from group B
ordersFilteredB = (orders_no_outliers[orders_no_outliers['group']=='B']
                  .groupby('visitorId', as_index=False)
                  .agg({'transactionId' : pd.Series.nunique})
                 )

ordersFilteredB.columns = ['visitorId', 'orders']

In [ ]:
# create sample for filtered group A and group B
## filtered Group A
sampleFilteredA = (pd.concat([ordersFilteredA['orders'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='A']['visitors'].sum()
                                                -len(ordersFilteredA['orders'])
                                               ),
                                name='orders'
                               )
                     ], axis=0
                    )
          )

## filtered Group B
sampleFilteredB = (pd.concat([ordersFilteredB['orders'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='B']['visitors'].sum()
                                                -len(ordersFilteredB['orders'])
                                               ),
                                name='orders'
                               )
                     ], axis=0)
          )

In [ ]:
# apply the two-proportion Z-test
proportion_test(sampleFilteredA, sampleFilteredB, 0.05)

In [ ]:
# apply mannwhitneyu test
mannwhitneyu_test(sampleFilteredA, sampleFilteredB, 0.05)

**INSIGHT**<br>
- From the raw dataset analysis, we found that the conversion rates of groups A and B are significantly different, where the conversion rate in group B is 15.2% higher than group A 
- The filtered dataset analysis has the same result as the raw dataset analysis.
- However, the relative difference in the conversion rate of group B is only 18% better than group A

#### Average Order Size

1. Comparing the average order size between the groups using the raw data
2. Comparing the average order size between the groups using the filtered data

##### Raw Data

In [ ]:
# Seperate `orders` from group A
revenueUsersA = (orders_clean[orders_clean['group']=='A']
                  .groupby('visitorId', as_index=False)
                  .agg({'revenue' : 'sum'})
                 )

revenueUsersA.columns = ['visitorId', 'revenue']

#----------------------------------------------------------------------------------
# Seperate `orders` from group B
revenueUsersB = (orders_clean[orders_clean['group']=='B']
                  .groupby('visitorId', as_index=False)
                  .agg({'revenue' : 'sum'})
                 )

revenueUsersB.columns = ['visitorId', 'revenue']

<div class="alert alert-block alert-warning">
<b>Reviewer's comment v1</b> <a class="tocSkip"></a>
    
You do not have to aggregate the `revenue` before calculating the statistical significance for the **average order size**. <br> Aggregating by `visitor_id` loses that level of detailed required by the statistical model to check the actual revenue distribution.

Using the revenue without aggregation is enough to provide the required solution.
</div>

In [ ]:
print('Total rows revenue A:', len(revenueUsersA))
print(revenueUsersA.head())

print('Total rows revenue B:', len(revenueUsersB))
print(revenueUsersB.head())

In [ ]:
# create sample for group A and group B
## Group A
sampleRevenueA = (pd.concat([revenueUsersA['revenue'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='A']['visitors'].sum()
                                                -len(revenueUsersA['revenue'])
                                               ),
                                name='revenue'
                               )
                     ], axis=0
                    )
          )

## Group B
sampleRevenueB = (pd.concat([revenueUsersB['revenue'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='B']['visitors'].sum()
                                                -len(revenueUsersB['revenue'])
                                               ),
                                name='revenue'
                               )
                     ], axis=0)
          )

In [ ]:
# apply the normality test
shapiro_wilk_test(sampleRevenueA, sampleRevenueB, 0.05)

**Note:** We will use the Mann-Whitney U test since the distribution of samples are not normal

In [ ]:
# apply the Mann Whitney U test
mannwhitneyu_test(sampleRevenueA, sampleRevenueB, 0.05)

##### Filtered Data

In [ ]:
# Seperate `orders_no_outliers` from group A
filteredRevenueA = (orders_no_outliers[orders_no_outliers['group']=='A']
                    .groupby('visitorId', as_index=False)
                    .agg({'revenue' : 'sum'})
                    )

filteredRevenueA.columns = ['visitorId', 'revenue']

#----------------------------------------------------------------------------------
# Seperate `orders_no_outliers` from group B
filteredRevenueB = (orders_no_outliers[orders_no_outliers['group']=='B']
                    .groupby('visitorId', as_index=False)
                    .agg({'revenue' : 'sum'})
                   )

filteredRevenueB.columns = ['visitorI', 'revenue']

<div class="alert alert-block alert-warning">
<b>Reviewer's comment v1</b> <a class="tocSkip"></a>
    
You do not have to aggregate the `revenue` before calculating the statistical significance for the **average order size**. <br> Aggregating by `visitor_id` loses that level of detailed required by the statistical model to check the actual revenue distribution.

Using the revenue without aggregation is enough to provide the required solution.
</div>

In [ ]:
print('Total rows filtered revenue A:', len(filteredRevenueA))
print(filteredRevenueA.head())

print('Total rows filtered revenue B:', len(filteredRevenueB))
print(filteredRevenueB.head())

In [ ]:
# create sample for filtered group A and group B
## Filtered Group A
sampleFilteredRevenueA = (pd.concat([filteredRevenueA['revenue'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='A']['visitors'].sum()
                                                -len(filteredRevenueA['revenue'])
                                               ),
                                name='revenue'
                               )
                     ], axis=0
                    )
          )

## Filtered Group B
sampleFilteredRevenueB = (pd.concat([filteredRevenueB['revenue'],
                      pd.Series(0,
                                index=np.arange(visitors[visitors['group']=='B']['visitors'].sum()
                                                -len(filteredRevenueB['revenue'])
                                               ),
                                name='revenue'
                               )
                     ], axis=0)
          )

In [ ]:
# normality test
shapiro_wilk_test(sampleFilteredRevenueA, sampleFilteredRevenueB, 0.05)

In [ ]:
# distribution test
mannwhitneyu_test(sampleFilteredRevenueA, sampleFilteredRevenueB, 0.05)

**INSIGHT**<br>
- From the raw dataset analysis, we found that the average order sizes of groups A and B are significantly different, where the average order size in group B is 44.7% higher than group A 
- The filtered dataset analysis has the same result as the raw dataset analysis.
- However, the relative difference in average order size is lower. The average order size in group B is only 14.4% better than group A

<div class="alert alert-block alert-success">
<b>Reviewer's comment v1</b> <a class="tocSkip"></a>

Great Job Ica. <br> Your statistical significance test has been on point and very clear so far. <br> 

</div>


<div class="alert alert-block alert-warning">
<b>Reviewer's comment v1</b> <a class="tocSkip"></a>
    
Your conclusion might change where neccessary based on result from the implemented code change
</div>

## Decision Result

After examining the cumulative changes in revenue, average order size, and conversion rate, we have determined that Group B (the treatment group) consistently outperforms Group A (the control group). This trend has been observed over the course of one month, during which both groups have stabilized noticeably.<br>

To validate our findings, we have conducted a hypothesis test on both the raw dataset and the filtered dataset (without outliers) to determine if there is a significant difference in revenue and conversion rate between the two groups. Our results indicate that Group B is significantly higher than Group A across all metrics. Although outliers have caused some metrics for Group B to be overvalued, our findings demonstrate that the treatment group is still performing better overall.<br>

Based on these results, we have decided to end the A/B testing and conclude that the treatment group (B) is better to the control group (A) in terms of generating higher revenue and conversion rates.